# Create Maps

In [ ]:
import os
import folium
import pandas as pd
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point
from folium.plugins import FastMarkerCluster

## **Dataframe** Analises

In [ ]:
# Tabela Pontos de Monitoramento
df = pd.read_csv(
    os.path.join('data', 'tab_analises.csv'),
)

# Ajuste Dates
df['data_coleta']          = pd.to_datetime(df['data_coleta'])
df['hora_coleta']          = pd.to_datetime(df['hora_coleta'], format='%H:%M:%S').dt.time
df['data_inicio_operacao'] = pd.to_datetime(df['data_inicio_operacao'])
df['data_fim_operacao']    = pd.to_datetime(df['data_fim_operacao'])
df['data_inicio_query']    = pd.to_datetime(df['data_inicio_query'])
df['data_fim_query']       = pd.to_datetime(df['data_fim_query'])

# Dtypes
print(df.dtypes)

# Results
df.head()

## Colunas

In [ ]:
columns = list(df.columns)
print(columns)
columns = columns[0:4] + columns[11:]
columns = columns[0:9] + columns[11:]
columns

## Group By

In [ ]:
# Group
df = pd.DataFrame(df.groupby(columns, dropna=False)['cod_ponto'].count())
df = df.rename(columns=lambda x: x.replace('cod_ponto', 'n_amostras'))
df = df.reset_index()

# Seleciona os que tem coordenadas
df = df[(df['longitude_dd'] != 0) & (df['latitude_dd'] != 0)]

# Results
df

## Cria *Geodataframe*

In [ ]:
# Create Geometry
geometry = [Point(xy) for xy in zip(df['longitude_dd'], df['latitude_dd'])]

# Create Geodataframe
gdf = gpd.GeoDataFrame(
    df,
    crs='EPSG:4326',
    geometry=geometry
)

In [ ]:
# Transform Coordenate and Format
gjson = gdf.to_crs(epsg='4326').to_json()

In [ ]:
# Cria o mapa
m = folium.Map(
    location=[-23.9619271,-46.3427499],
    zoom_start=12,
)

callback = (
    'function (row) {'
    'var circle = L.circle(new L.LatLng(row[0], row[1]), {color: "red",  radius: 10000});'
    'return circle};'
)

m.add_child(
    FastMarkerCluster(
        df[['latitude_dd', 'longitude_dd']].values.tolist(),
        #callback=callback,
    )
)

# add all the point from the file to the map object using FastMarkerCluster
m.add_child(FastMarkerCluster(df['latitude_dd'], df['longitude_dd']))
# save the map
#m.save("save_file.html")
m

In [ ]:
# Cria o mapa
m = folium.Map(
    location=[-23.9619271,-46.3427499],
    zoom_start=12,
    #width=767,
    #height=460,
)

In [ ]:
shp = folium.features.GeoJson(gjson)
m.add_child(shp)
m.fit_bounds(m.get_bounds())

In [ ]:
# Column with category
col_categories = 'sistema_hidrico'

# Set palette
palette_polygon = 'Paired'

# Get list of unique values
categories = set(df[col_categories])
categories = list(categories)
categories.sort()

# See the palette chosed
pal = sns.color_palette(palette_polygon, n_colors=len(categories))

# Set dictionary
colors = dict(zip(categories, pal.as_hex()))

In [ ]:
# Cria o mapa
m = folium.Map(
    location=[-23.9619271,-46.3427499],
    zoom_start=12,
    #width=767,
    #height=460,
)

# Adiciona as diferentes empresas com cores por bairros
for index, row in df.iterrows():
    if row['sistema_hidrico'] in colors.keys():
        folium.Marker(
            location=[row['latitude_dd'], row['longitude_dd']],
            #popup=row['sistema_hidrico'],
            #tooltip=row['localizacao'],
            #icon=folium.Icon(color=colors[row[col_categories]], icon='leaf')
        ).add_to(m)

os.makedirs('maps', exist_ok=True)
m.save(os.path.join('maps', 'map.html'))
# Apresenta o mapa
#m

## Results

In [ ]:
# Save
os.makedirs('maps', exist_ok=True)
m.save(os.path.join('maps', 'map.html'))

# Apresenta o mapa
m

## Outros

In [ ]:
# Dictonary to Rename
dict_columns = {
    'Cód.Ponto'   : 'cod_ponto',
    'Sist.Hidrico': 'sistema_hidrico',
    'Localização' : 'localizacao',
    'Data Início' : 'data_inicio',
    'Data Fim'    : 'data_fim',
    'Município'   : 'municipio',
}

# Rename e Reorder
df = df.rename(columns=dict_columns)
df = df[[v for k, v in dict_columns.items()]]

# Columns
print(list(df.columns))

# Results
df.head()